In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [2]:
# Check for GPU
import tensorflow as tf
try:
    from google.colab import drive
    IN_COLAB=True
except:
    IN_COLAB=False

if IN_COLAB:
    print("We're running Colab")
else:
    print(tf.config.list_physical_devices())
    print('\nCUDA GPU: ' + str(tf.test.is_gpu_available(cuda_only=True)))

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.

CUDA GPU: False


# Data Preprocessing

In [3]:
import os
os.chdir('..')
df = pd.read_csv('./daily01-ithaca/daily01-NY_Ithaca_13_E.csv', header = 0, index_col = 0)
Date = pd.to_datetime(df.LST_DATE, format='%Y%m%d', errors='coerce')
df['Time'] = Date

In [4]:
df.head()

WBANNO  LST_DATE  CRX_VN  LONGITUDE  LATITUDE  T_DAILY_MAX  T_DAILY_MIN  \
0   64758  20041027   1.201     -76.25     42.44          NaN          NaN   
1   64758  20041028   1.201     -76.25     42.44         12.7         -0.3   
2   64758  20041029   1.201     -76.25     42.44         16.3          2.5   
3   64758  20041030   1.201     -76.25     42.44         17.5         10.5   
4   64758  20041031   1.201     -76.25     42.44         17.0          9.1   

   T_DAILY_MEAN  T_DAILY_AVG  P_DAILY_CALC  ...  SOIL_MOISTURE_10_DAILY  \
0           NaN          NaN           NaN  ...                     NaN   
1           6.2          5.0           0.0  ...                     NaN   
2           9.4          9.7           0.0  ...                     NaN   
3          14.0         14.5           1.8  ...                     NaN   
4          13.1         12.6           0.0  ...                     NaN   

  SOIL_MOISTURE_20_DAILY  SOIL_MOISTURE_50_DAILY  SOIL_MOISTURE_100_DAILY  \
0                    NaN                     NaN                      NaN   
1                    NaN                     NaN                      NaN   
2                    NaN                     NaN                      NaN   
3                    NaN                     NaN                      NaN   
4                    NaN                     NaN                      NaN   

   SOIL_TEMP_5_DAILY  SOIL_TEMP_10_DAILY  SOIL_TEMP_20_DAILY  \
0                NaN                 NaN                 NaN   
1                NaN                 NaN                 NaN   
2                NaN                 NaN                 NaN   
3                NaN                 NaN                 NaN   
4                NaN                 NaN                 NaN   

   SOIL_TEMP_50_DAILY  SOIL_TEMP_100_DAILY       Time  
0                 NaN                  NaN 2004-10-27  
1                 NaN                  NaN 2004-10-28  
2                 NaN                  NaN 2004-10-29  
3                 NaN                  NaN 2004-10-30  
4                 NaN                  NaN 2004-10-31  

[5 rows x 29 columns]

In [5]:
data = df[['T_DAILY_MAX',
       'T_DAILY_MIN', 'T_DAILY_MEAN', 'T_DAILY_AVG', 'P_DAILY_CALC',
       'SOLARAD_DAILY', 'SUR_TEMP_DAILY_MAX',
       'SUR_TEMP_DAILY_MIN', 'SUR_TEMP_DAILY_AVG', 'RH_DAILY_MAX',
       'RH_DAILY_MIN', 'RH_DAILY_AVG']]
data.index = df['Time']

In [6]:
data.shape

(6949, 12)

In [7]:
# forward fill the missing values
data.ffill(axis = 0, inplace = True)
# drop NaN at the top
data.dropna(inplace = True)
# set target
data['target'] = data['T_DAILY_AVG']
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2, shuffle = False)

In [8]:
from sklearn.preprocessing import MinMaxScaler
import pickle

scaler = MinMaxScaler()
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

In [9]:
train.shape

(3672, 13)

In [10]:
test.shape

(918, 13)

In [11]:
# splitting data into sequences
def split_sequences(features, target, seq_len, forecast_len):
    X,y = list(), list()
    for i in range(len(features)):
        end_input = i + seq_len
        end_predict = end_input + forecast_len
        if end_predict > len(features)-1:
            break
        seq_x, seq_y = features[i:end_input,:], target[end_input:end_predict]
        X.append(seq_x)
        y.append(seq_y)
    return tf.convert_to_tensor(X, dtype=tf.float64), tf.convert_to_tensor(y, dtype=tf.float64)

# Define Model

In [12]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, RNN, LSTMCell, Input, Bidirectional
from tensorflow.keras.losses import BinaryCrossentropy, MeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import plot_model

class MyModel(tf.keras.Model):

    def __init__(self, input_shape, output_shape, name = 'BiLSTM-FC'):
        super().__init__(name = name)
        self.input_layer = Input(shape = input_shape, name = 'input')
        self.lstm1 = Bidirectional(LSTM(units=30, activation = 'tanh', input_shape = input_shape, return_sequences=False, name = 'lstm_1'),name = 'bilstm_1')
        self.dense1 = Dense(units=output_shape, activation = 'sigmoid', name = 'dense_1')
        #self.dropout = tf.keras.layers.Dropout(0.5)

    def call(self, inputs, training=False):
        x = self.lstm1(inputs)
        x = self.dense1(x)
        #if training:
        #  x = self.dropout(x, training=training)
        return x

    def summary(self):
        model = Model(inputs = [self.input_layer], outputs = self.call(self.input_layer), name = self.name)
        return model.summary()

# Model Training
## input length : output length = 16:4

In [13]:
# prepare sequences
seq_len = 16
forecast_len = 4
X_train, y_train = split_sequences(train[:,:-1], train[:,-1], seq_len = seq_len, forecast_len = forecast_len)
X_test, y_test = split_sequences(test[:,:-1], test[:,-1],seq_len = seq_len, forecast_len =  forecast_len)
n_features = X_train.shape[2]

In [14]:
X_train.shape

TensorShape([3652, 16, 12])

In [15]:
y_train.shape

TensorShape([3652, 4])

In [16]:
# create model instance
model_name = 'BiLSTM_16-4'
model = MyModel(input_shape = (seq_len, n_features), output_shape = (forecast_len), name = model_name)
model.summary()

Model: "BiLSTM_16-4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 16, 12)]          0         
                                                                 
 bilstm_1 (Bidirectional)    (None, 60)                10320     
                                                                 
 dense_1 (Dense)             (None, 4)                 244       
                                                                 
Total params: 10,564
Trainable params: 10,564
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Fit the model
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate = 0.01), metrics = ['mse', 'acc'])
model.fit(X_train,
          y_train,
          batch_size=100,
          epochs=30,
          verbose='auto',
          callbacks=None,
          validation_split=0.1,
          shuffle=True)
# traning for > 40 epoch starts to overfit
# save trained model
model.save('./LSTM/models_daily_v2/' + model_name)

Epoch 1/30
33/33 [==============================] - 3s 24ms/step - loss: 0.0187 - mse: 0.0187 - acc: 0.2419 - val_loss: 0.0082 - val_mse: 0.0082 - val_acc: 0.2404
Epoch 2/30
33/33 [==============================] - 0s 7ms/step - loss: 0.0084 - mse: 0.0084 - acc: 0.2626 - val_loss: 0.0073 - val_mse: 0.0073 - val_acc: 0.2923
Epoch 3/30
33/33 [==============================] - 0s 7ms/step - loss: 0.0079 - mse: 0.0079 - acc: 0.3083 - val_loss: 0.0067 - val_mse: 0.0067 - val_acc: 0.2541
Epoch 4/30
33/33 [==============================] - 0s 8ms/step - loss: 0.0078 - mse: 0.0078 - acc: 0.3357 - val_loss: 0.0066 - val_mse: 0.0066 - val_acc: 0.2623
Epoch 5/30
33/33 [==============================] - 0s 7ms/step - loss: 0.0076 - mse: 0.0076 - acc: 0.3421 - val_loss: 0.0070 - val_mse: 0.0070 - val_acc: 0.3852
Epoch 6/30
33/33 [==============================] - 0s 6ms/step - loss: 0.0075 - mse: 0.0075 - acc: 0.3475 - val_loss: 0.0064 - val_mse: 0.0064 - val_acc: 0.3197
Epoch 7/30
33/33 [=========

INFO:tensorflow:Assets written to: ./LSTM/models_daily_v2/BiLSTM_16-4\assets


INFO:tensorflow:Assets written to: ./LSTM/models_daily_v2/BiLSTM_16-4\assets


In [18]:
y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)

29/29 [==============================] - 0s 1ms/step


In [19]:
from sklearn.metrics import mean_squared_error
print('mean_squared_error')
print('train set:', mean_squared_error(y_train, y_hat_train, sample_weight=None))
print('test set:', mean_squared_error(y_test, y_hat_test, sample_weight=None))

mean_squared_error
train set: 0.006953973749311129
test set: 0.007176994875978931


## input length : output length = 24:6

In [20]:
# reset memory
tf.Graph().as_default()

# prepare sequences
seq_len = 24
forecast_len = 6
X_train, y_train = split_sequences(train[:,:-1], train[:,-1], seq_len = seq_len, forecast_len = forecast_len)
X_test, y_test = split_sequences(test[:,:-1], test[:,-1],seq_len = seq_len, forecast_len =  forecast_len)
n_features = X_train.shape[2]

In [21]:
# create model instance
model_name = 'BiLSTM_24-6'
model = MyModel(input_shape = (seq_len, n_features), output_shape = (forecast_len), name = model_name)
model.summary()

Model: "BiLSTM_24-6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 24, 12)]          0         
                                                                 
 bilstm_1 (Bidirectional)    (None, 60)                10320     
                                                                 
 dense_1 (Dense)             (None, 6)                 366       
                                                                 
Total params: 10,686
Trainable params: 10,686
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Fit the model
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate = 0.01), metrics = ['mse', 'acc'])
model.fit(X_train,
          y_train,
          batch_size=100,
          epochs=30,
          verbose='auto',
          callbacks=None,
          validation_split=0.1,
          shuffle=True)
# start to overfit for epoch > 40

Epoch 1/30
33/33 [==============================] - 3s 37ms/step - loss: 0.0155 - mse: 0.0155 - acc: 0.1709 - val_loss: 0.0082 - val_mse: 0.0082 - val_acc: 0.1342
Epoch 2/30
33/33 [==============================] - 0s 12ms/step - loss: 0.0087 - mse: 0.0087 - acc: 0.1633 - val_loss: 0.0077 - val_mse: 0.0077 - val_acc: 0.1945
Epoch 3/30
33/33 [==============================] - 0s 12ms/step - loss: 0.0086 - mse: 0.0086 - acc: 0.1852 - val_loss: 0.0077 - val_mse: 0.0077 - val_acc: 0.1918
Epoch 4/30
33/33 [==============================] - 0s 12ms/step - loss: 0.0085 - mse: 0.0085 - acc: 0.1913 - val_loss: 0.0076 - val_mse: 0.0076 - val_acc: 0.1808
Epoch 5/30
33/33 [==============================] - 0s 12ms/step - loss: 0.0081 - mse: 0.0081 - acc: 0.2157 - val_loss: 0.0077 - val_mse: 0.0077 - val_acc: 0.1616
Epoch 6/30
33/33 [==============================] - 0s 12ms/step - loss: 0.0081 - mse: 0.0081 - acc: 0.2359 - val_loss: 0.0075 - val_mse: 0.0075 - val_acc: 0.2137
Epoch 7/30
33/33 [====

In [23]:
# evaluate the model
# save trained model
model.save('./LSTM/models_daily_v2/' + model_name)
from sklearn.metrics import mean_squared_error

y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)

print('mean_squared_error')
print('train set:', mean_squared_error(y_train, y_hat_train, sample_weight=None))
print('test set:', mean_squared_error(y_test, y_hat_test, sample_weight=None))

INFO:tensorflow:Assets written to: ./LSTM/models_daily_v2/BiLSTM_24-6\assets


INFO:tensorflow:Assets written to: ./LSTM/models_daily_v2/BiLSTM_24-6\assets


28/28 [==============================] - 0s 1ms/step
mean_squared_error
train set: 0.006723440860645701
test set: 0.007367470615396028


## input length : output length = 32:8

In [24]:
# reset memory
tf.Graph().as_default()

# prepare sequences
seq_len = 32
forecast_len = 8
X_train, y_train = split_sequences(train[:,:-1], train[:,-1], seq_len = seq_len, forecast_len = forecast_len)
X_test, y_test = split_sequences(test[:,:-1], test[:,-1],seq_len = seq_len, forecast_len =  forecast_len)
n_features = X_train.shape[2]

In [25]:
# create model instance
model_name = 'BiLSTM_32-8'
model = MyModel(input_shape = (seq_len, n_features), output_shape = (forecast_len), name = model_name)
model.summary()

Model: "BiLSTM_32-8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 32, 12)]          0         
                                                                 
 bilstm_1 (Bidirectional)    (None, 60)                10320     
                                                                 
 dense_1 (Dense)             (None, 8)                 488       
                                                                 
Total params: 10,808
Trainable params: 10,808
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Fit the model
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate = 0.01), metrics = ['mse', 'acc'])
model.fit(X_train,
          y_train,
          batch_size=100,
          epochs=30,
          verbose='auto',
          callbacks=None,
          validation_split=0.1,
          shuffle=True)
# also overfit if epoch > 40

Epoch 1/30
33/33 [==============================] - 3s 30ms/step - loss: 0.0182 - mse: 0.0182 - acc: 0.1245 - val_loss: 0.0091 - val_mse: 0.0091 - val_acc: 0.1593
Epoch 2/30
33/33 [==============================] - 0s 10ms/step - loss: 0.0091 - mse: 0.0091 - acc: 0.1313 - val_loss: 0.0080 - val_mse: 0.0080 - val_acc: 0.1291
Epoch 3/30
33/33 [==============================] - 0s 12ms/step - loss: 0.0088 - mse: 0.0088 - acc: 0.1475 - val_loss: 0.0081 - val_mse: 0.0081 - val_acc: 0.1593
Epoch 4/30
33/33 [==============================] - 0s 10ms/step - loss: 0.0087 - mse: 0.0087 - acc: 0.1447 - val_loss: 0.0082 - val_mse: 0.0082 - val_acc: 0.1374
Epoch 5/30
33/33 [==============================] - 0s 10ms/step - loss: 0.0084 - mse: 0.0084 - acc: 0.1539 - val_loss: 0.0078 - val_mse: 0.0078 - val_acc: 0.1621
Epoch 6/30
33/33 [==============================] - 0s 10ms/step - loss: 0.0083 - mse: 0.0083 - acc: 0.1714 - val_loss: 0.0079 - val_mse: 0.0079 - val_acc: 0.1896
Epoch 7/30
33/33 [====

In [27]:
# save trained model
model.save('./LSTM/models_daily_v2/' + model_name)
# evaluate the model
from sklearn.metrics import mean_squared_error

y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)

print('mean_squared_error')
print('train set:', mean_squared_error(y_train, y_hat_train, sample_weight=None))
print('test set:', mean_squared_error(y_test, y_hat_test, sample_weight=None))

INFO:tensorflow:Assets written to: ./LSTM/models_daily_v2/BiLSTM_32-8\assets


INFO:tensorflow:Assets written to: ./LSTM/models_daily_v2/BiLSTM_32-8\assets


28/28 [==============================] - 0s 2ms/step
mean_squared_error
train set: 0.0068966127110462435
test set: 0.007929253874674594


## input length : output length = 40:10

In [28]:
# reset memory
tf.Graph().as_default()

# prepare sequences
seq_len = 40
forecast_len = 10
X_train, y_train = split_sequences(train[:,:-1], train[:,-1], seq_len = seq_len, forecast_len = forecast_len)
X_test, y_test = split_sequences(test[:,:-1], test[:,-1],seq_len = seq_len, forecast_len =  forecast_len)
n_features = X_train.shape[2]

In [29]:
# create model instance
model_name = 'BiLSTM_40-10'
model = MyModel(input_shape = (seq_len, n_features), output_shape = (forecast_len), name = model_name)
model.summary()

Model: "BiLSTM_40-10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 40, 12)]          0         
                                                                 
 bilstm_1 (Bidirectional)    (None, 60)                10320     
                                                                 
 dense_1 (Dense)             (None, 10)                610       
                                                                 
Total params: 10,930
Trainable params: 10,930
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Fit the model
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate = 0.01), metrics = ['mse', 'acc'])
model.fit(X_train,
          y_train,
          batch_size=100,
          epochs=30,
          verbose='auto',
          callbacks=None,
          validation_split=0.1,
          shuffle=True)

Epoch 1/30
33/33 [==============================] - 4s 37ms/step - loss: 0.0149 - mse: 0.0149 - acc: 0.0963 - val_loss: 0.0093 - val_mse: 0.0093 - val_acc: 0.0937
Epoch 2/30
33/33 [==============================] - 1s 22ms/step - loss: 0.0093 - mse: 0.0093 - acc: 0.1074 - val_loss: 0.0085 - val_mse: 0.0085 - val_acc: 0.1019
Epoch 3/30
33/33 [==============================] - 1s 22ms/step - loss: 0.0089 - mse: 0.0089 - acc: 0.1218 - val_loss: 0.0101 - val_mse: 0.0101 - val_acc: 0.1267
Epoch 4/30
33/33 [==============================] - 1s 22ms/step - loss: 0.0088 - mse: 0.0088 - acc: 0.1405 - val_loss: 0.0080 - val_mse: 0.0080 - val_acc: 0.1074
Epoch 5/30
33/33 [==============================] - 1s 22ms/step - loss: 0.0085 - mse: 0.0085 - acc: 0.1329 - val_loss: 0.0085 - val_mse: 0.0085 - val_acc: 0.1377
Epoch 6/30
33/33 [==============================] - 1s 22ms/step - loss: 0.0084 - mse: 0.0084 - acc: 0.1513 - val_loss: 0.0077 - val_mse: 0.0077 - val_acc: 0.1460
Epoch 7/30
33/33 [====

In [31]:
# save trained model
model.save('./LSTM/models_daily_v2/' + model_name)
# evaluate the model
from sklearn.metrics import mean_squared_error

y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)

print('mean_squared_error')
print('train set:', mean_squared_error(y_train, y_hat_train, sample_weight=None))
print('test set:', mean_squared_error(y_test, y_hat_test, sample_weight=None))

INFO:tensorflow:Assets written to: ./LSTM/models_daily_v2/BiLSTM_40-10\assets


INFO:tensorflow:Assets written to: ./LSTM/models_daily_v2/BiLSTM_40-10\assets


28/28 [==============================] - 0s 2ms/step
mean_squared_error
train set: 0.006607562358788653
test set: 0.008242106491878513
